In [49]:
import os 
cwd = os.getcwd()
print(cwd)

C:\Users\anish\OneDrive\Documents\GitHub\data-analytics-PYTHON


### **Step-by-Step Summary for Implementing the Project**  

---

### **1️⃣ Set Up the Environment**  
✅ **Import necessary Python packages**  
- `os` → To navigate files and directories  
- `json` → To work with JSON files  
- `csv` → To read and write CSV files  

✅ **Ensure files are loaded correctly**  
- If using **Google Colab**, upload files manually or mount Google Drive  
- If using **Jupyter Notebook**, ensure files are in the correct directory  

---

### **2️⃣ Load the Data**  
Create a function `load_data()` to:  
- Load sales data from `sales_data.csv` into a dictionary (`sales_data`)  
- Load product details from JSON files in the `product_details` folder into a dictionary (`product_details`)  
- Load product descriptions from text files in the `product_descriptions` folder into a dictionary (`product_descriptions`)  

---

### **3️⃣ Add/Update Product Information**  
Create functions to allow the admin to add or update data:

✅ **`update_sales_data()`**  
- Add/update **sales data** for a given SKU  

✅ **`update_product_details()`**  
- Add/update **product details** for a given SKU  

✅ **`update_product_description()`**  
- Add/update **product description** for a given SKU  

✅ **`update()`** (Main function to update all data)  
- Take **user input** for SKU, sales data, product details, and description  
- **Validate inputs**  
  - SKU must be **exactly 13 characters**  
  - Sales data must have **14 whole numbers**  
  - Product details and description are required  
- If inputs are **valid**, update all three dictionaries  
- Print a **success message**  
- Return updated dictionaries  

---

### **4️⃣ Overwrite and Save Data to Files**  
Create `dump_data()` function to:  
- **Write `sales_data` to `sales_data.csv`**  
- **Save `product_details` as JSON files** in the `product_details` folder  
- **Save `product_descriptions` as text files** in the `product_descriptions` folder  
- If folders **don’t exist, create them automatically**  

---

### **5️⃣ Final Execution**  
✅ Load existing data  
✅ Call `update()` to modify product info  
✅ Call `dump_data()` to save changes to files  

---

### **Outcome**  
📌 **Admin can:**
- Load product data ✅  
- Add new products ✅  
- Update existing product information ✅  
- Ensure data integrity ✅  
- Overwrite files safely ✅  

In [52]:
import os
import json
import pandas as pd
import re

# File paths
csv_file = "C:\\Users\\anish\\OneDrive\\Documents\\GitHub\\mainfolder\\sales_data.csv"
json_folder = "C:\\Users\\anish\\OneDrive\\Documents\\GitHub\\mainfolder\\product_details"
txt_folder = "C:\\Users\\anish\\OneDrive\\Documents\\GitHub\\mainfolder\\product_descriptions"

def load_sales_data(csv_file):
    """Load sales data from a CSV file and return as a DataFrame."""
    if os.path.exists(csv_file):
        return pd.read_csv(csv_file)
    else:
        return pd.DataFrame(columns=["Product_SKU"] + [f"Day{i}" for i in range(1, 15)])

def load_product_details(json_folder):
    """Load product details from JSON files."""
    product_details = {}
    if os.path.exists(json_folder):
        for filename in os.listdir(json_folder):
            if filename.startswith("details_") and filename.endswith('.json'):
                sku = filename.replace("details_", "").replace('.json', '')
                with open(os.path.join(json_folder, filename), 'r', encoding='utf-8') as file:
                    product_details[sku] = json.load(file)
    return product_details

def load_product_descriptions(txt_folder):
    """Load product descriptions from text files."""
    product_descriptions = {}
    if os.path.exists(txt_folder):
        for filename in os.listdir(txt_folder):
            if filename.startswith("description_") and filename.endswith('.txt'):
                sku = filename.replace("description_", "").replace('.txt', '')
                with open(os.path.join(txt_folder, filename), 'r', encoding='utf-8') as file:
                    product_descriptions[sku] = file.read().strip()
    return product_descriptions

def load_data():
    """Load all required data files into structured formats."""
    sales_data = load_sales_data(csv_file)
    product_details = load_product_details(json_folder)
    product_descriptions = load_product_descriptions(txt_folder)
    return sales_data, product_details, product_descriptions

# Load initial data
sales_data, product_details, product_descriptions = load_data()
print("✅ Data Loaded Successfully")


✅ Data Loaded Successfully


In [54]:
def validate_sku(sku):
    """Validate SKU: It should be exactly 13 characters, alphanumeric, and uppercase."""
    return bool(re.fullmatch(r"[A-Z0-9]{13}", sku))

def validate_sales_data(data):
    """Validate sales data: It should be 14 whole numbers separated by spaces."""
    numbers = data.split()
    return len(numbers) == 14 and all(num.isdigit() for num in numbers)

def update_sales_data():
    """Update or add sales data for a product SKU."""
    global sales_data
    
    sku = input("Enter Product Sales SKU: ").strip()
    if not validate_sku(sku):
        print("❌ Invalid SKU! It should be exactly 13 characters, alphanumeric, and uppercase.")
        return
    
    if sku in sales_data["Product_SKU"].values:
        choice = input("This SKU ID is found. Do you want to update the data? (yes/no): ").strip().lower()
        if choice != 'yes':
            return
        day = input("Enter specific day to update (1-14) or press Enter to update all days: ")
        if day:
            day = int(day)
            sales = int(input(f"Enter sales data for Day {day}: "))
            sales_data.loc[sales_data["Product_SKU"] == sku, f"Day{day}"] = sales
        else:
            sales_data_input = input("Enter sales data for all 14 days (separated by spaces): ")
            if not validate_sales_data(sales_data_input):
                print("❌ Invalid sales data! It should contain exactly 14 whole numbers separated by spaces.")
                return
            sales_list = list(map(int, sales_data_input.split()))
            for i in range(1, 15):
                sales_data.loc[sales_data["Product_SKU"] == sku, f"Day{i}"] = sales_list[i-1]
    else:
        choice = input("This SKU ID is not found. Do you want to add new data? (yes/no): ").strip().lower()
        if choice != 'yes':
            return
        sales_data_input = input("Enter sales data for all 14 days (separated by spaces): ")
        if not validate_sales_data(sales_data_input):
            print("❌ Invalid sales data! It should contain exactly 14 whole numbers separated by spaces.")
            return
        new_row = {"Product_SKU": sku}
        sales_list = list(map(int, sales_data_input.split()))
        for i in range(1, 15):
            new_row[f"Day{i}"] = sales_list[i-1]
        sales_data = pd.concat([sales_data, pd.DataFrame([new_row])], ignore_index=True)

    print("✅ Sales data updated successfully!")

def update_product_details():
    """Update or add product details in JSON format."""
    sku = input("Enter Product Detail SKU: ").strip()
    if not validate_sku(sku):
        print("❌ Invalid SKU!")
        return
    
    details = {
        "product_name": input("Enter Product Name: "),
        "brand": input("Enter Brand: "),
        "model": input("Enter Model: "),
        "specifications": input("Enter Specifications: "),
        "price": input("Enter Price: "),
        "availability": input("Enter Availability: ")
    }
    
    product_details[sku] = details
    print("✅ Product details updated successfully!")

def update_product_description():
    """Update or add product description in a text file."""
    sku = input("Enter Product Description SKU: ").strip()
    if not validate_sku(sku):
        print("❌ Invalid SKU!")
        return
    
    product_descriptions[sku] = input("Enter Product Description: ")
    print("✅ Product description updated successfully!")


def update():
    """Calls all update functions for sales, product details, and descriptions."""
    update_sales_data()
    update_product_details()
    update_product_description()

# Call update function when needed
print("🔄 Ready to update data")


🔄 Ready to update data


In [56]:
def dump_data():
    """Dump updated data into files, ensuring the correct folder structure."""
    
    # Define main folder path
    main_folder = "C:\\Users\\anish\\OneDrive\\Documents\\GitHub\\mainfolder"

    # Ensure main folder exists
    os.makedirs(main_folder, exist_ok=True)
    
    # Save sales data
    sales_data.to_csv(os.path.join(main_folder, "sales_data.csv"), index=False)
    print("✅ Sales data successfully written to CSV.")

    # Ensure subfolders exist
    product_details_folder = os.path.join(main_folder, "product_details")
    product_descriptions_folder = os.path.join(main_folder, "product_descriptions")
    os.makedirs(product_details_folder, exist_ok=True)
    os.makedirs(product_descriptions_folder, exist_ok=True)

    # Save product details
    for sku, details in product_details.items():
        with open(os.path.join(product_details_folder, f"details_{sku}.json"), "w", encoding="utf-8") as json_file:
            json.dump(details, json_file, indent=4)
    print("✅ Product details successfully written to JSON files.")

    # Save product descriptions
    for sku, description in product_descriptions.items():
        with open(os.path.join(product_descriptions_folder, f"description_{sku}.txt"), "w", encoding="utf-8") as txt_file:
            txt_file.write(description)
    print("✅ Product descriptions successfully written to text files.")

# Call dump_data() when needed
print("📂 Ready to dump data")


📂 Ready to dump data


In [60]:
def main ():
    
    load_data()
print('✅ Data Loaded Successfully')
up = input('Do you want to update the data :- (yes/no)').strip().lower()
if up == 'yes':
    update()
    print('update successfully')
    dump_data()
else:
    print('thank you !!!!!!')



✅ Data Loaded Successfully


Do you want to update the data :- (yes/no) no


thank you !!!!!!
